IMPORT UNSLOTH AND OTHER DEPENDENCIES


In [ ]:
!pip install unsloth
!pip install --upgrade --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers==0.0.25+814915d.d20230118" -f https://github.com/facebookresearch/xformers/releases/download/v0.0.25%2B814915d/wheels

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.7/164.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 602.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

IMPORTING THE MODEL

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.7: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 7.5. CUDA Toolkit = 12.4.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers, TRL and unsloth via:
`pip install --upgrade --no-cache-dir unsloth git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/trl.git`


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.10.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


IMPORTING DATASET

In [ ]:
!pip install datasets

from datasets import load_dataset

dataset = load_dataset("ClumsyAahDeveloper/sample003",split = "train",
)
print(dataset.column_names)
print(dataset)

README.md:   0%|          | 0.00/502 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/24.4k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/9.60k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/128 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/33 [00:00<?, ? examples/s]

['Question', 'Input', 'Instructions', 'Output']
Dataset({
    features: ['Question', 'Input', 'Instructions', 'Output'],
    num_rows: 128
})


In [ ]:
import pandas as pd

df = dataset.to_pandas()

pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)

df

,Question,Input,Instructions,Output
0,How do you implement a stack using an array?,Implementation of a stack using an array.,User query : How do you implement a stack usin...,Implement a stack using an array by maintainin...
1,"What is a truth table, and how is it used in l...",Truth table and its use in logic design.,"User query : What is a truth table, and how is...",A truth table is a tabular representation of a...
2,How can you detect and correct errors in logic...,Error detection and correction in logic circuits.,User query : How can you detect and correct er...,Error detection can be achieved through parity...
3,What are access modifiers in Java?,Explanation of access modifiers in Java.,User query : What are access modifiers in Java...,Access modifiers control the visibility of cla...
4,What is the Java Virtual Machine (JVM)?,Role of the Java Virtual Machine (JVM).,User query : What is the Java Virtual Machine ...,The JVM is an abstract computing machine that ...
5,What is method overriding?,Example of method overriding.,User query : What is method overriding? Input :,Method overriding allows a subclass to provide...
6,"What is a heap, and how is it structured?",Definition of a heap and its structure.,"User query : What is a heap, and how is it str...",A heap is a special tree-based data structure ...
7,"What are logic gates, and what are the basic t...",Logic gates and basic types.,"User query : What are logic gates, and what ar...",Logic gates are electronic components that per...
8,"Briefly explain the portable, secure and robus...","Features of Java: portability, security, robus...","User query : Briefly explain the portable, sec...",Java's features ensure code can run on any pla...
9,What is a Java thread?,Definition of a Java thread.,User query : What is a Java thread? Input :,A Java thread is a lightweight subprocess that...


In [ ]:
from huggingface_hub import login

login()

FORMATTING THE DATASET

In [ ]:
from unsloth import to_sharegpt
dataset = to_sharegpt(
    dataset,
    merged_prompt = "{Question}[[\nYour input is:\n{Input}]]",
    output_column_name = "Output",
    conversation_extension = 3,
)

Merging columns:   0%|          | 0/128 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/128 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/128 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/128 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/128 [00:00<?, ? examples/s]

Extending conversations:   0%|          | 0/128 [00:00<?, ? examples/s]

In [ ]:
from unsloth import standardize_sharegpt

dataset = standardize_sharegpt(dataset)

Standardizing format:   0%|          | 0/128 [00:00<?, ? examples/s]

PUTTING THE CHAT TEMPLATE

In [ ]:
chat_template = """Below are some instructions that describe some tasks. Write responses that appropriately completes the request.

### Instruction:
{INPUT}

### Response:
{OUTPUT}"""

from unsloth import apply_chat_template
dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
)

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/128 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd

def print_dataset(dataset):
    """Prints the dataset in a readable format."""
    if isinstance(dataset, pd.DataFrame):
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            print(dataset)
    else:
        try:
            print(dataset)
        except TypeError:
            print("Dataset is not printable directly. Try converting to DataFrame first.")


print_dataset(dataset)

Dataset({
    features: ['conversations', 'text'],
    num_rows: 128
})


In [ ]:
!pip install trl

SETTING TRAINING PARAMETERS

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
trainer = SFTTrainer(
    model=model,
    tokenizer = tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        num_train_epochs = 1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "wandb",
    ),

)

Map (num_proc=2):   0%|          | 0/128 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


TRAINING THE MODEL

In [ ]:
!pip install wandb

import wandb

wandb.init(project="my-sft-project",name="my-training-run")

trainer_stats = trainer.train()

wandb.finish()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers, TRL and Unsloth!
`pip install --upgrade --no-cache-dir unsloth git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/trl.git`


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 128 | Num Epochs = 4
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
1,2.017700
2,1.991100
3,2.039900
4,1.785100
5,1.741300
6,1.477800
7,1.331400
8,1.085800
9,1.040100
10,0.922100


train/epoch,▁▁▁▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█████
train/global_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇▇▇████
train/grad_norm,▆▅▆▅▂▆▄▃█▂▂▂▃▃▃▃▃▅▄▅▂▂▂▁▇▄▂▂▂▂▃▂▂▁▁▂▂▂▁▃
train/learning_rate,▂▄▅▇███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁
train/loss,███▇▆▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁
total_flos,4135260881584128.0
train/epoch,3.75
train/global_step,60
train/grad_norm,0.74778
train/learning_rate,0
train/loss,0.1333


CREATING AN INFERENCE

In [ ]:
FastLanguageModel.for_inference(model)
messages = [
    {"role": "user", "content": "indian prime minister"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Narendra Modi, elected in 2014 and again in 2019, is the current Prime Minister of India.<|end_of_text|>


SAVING THE FILE LOCALLY

In [ ]:
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

PUSHING IT TO THE HUGGINGFACE REPOSITORY

In [ ]:
model.push_to_hub("ClumsyAahDeveloper/lora_model",token = "")
tokenizer.push_to_hub("ClumsyAahDeveloper/lora_model",token = "")


README.md:   0%|          | 0.00/585 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/ClumsyAahDeveloper/lora_model


In [ ]:
!du -sh lora_model

169M	lora_model


RUNNING THE MODEL

In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model",
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)
pass

messages = [
    {"role": "user", "content": "Who is Haridev?"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

==((====))==  Unsloth 2024.10.7: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 7.5. CUDA Toolkit = 12.4.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Haridev is one of the main characters in the novel 'The Hunger Games', serving as a mentor to Katniss Everdeen in the 74th Hunger Games.<|end_of_text|>


SAVING IT TO OLLAMA

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
if True: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("ClumsyAahDeveloper/model", tokenizer, token = "")


if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("ClumsyAahDeveloper/model", tokenizer, quantization_method = "f16", token = "")

if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("ClumsyAahDeveloper/model", tokenizer, quantization_method = "q4_k_m", token = "")

if False:
    model.push_to_hub_gguf(
        "ClumsyAahDeveloper/model",
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.
Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.75 out of 12.67 RAM for saving.


100%|██████████| 32/32 [03:50<00:00,  7.19s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving model/pytorch_model-00001-of-00004.bin...
Unsloth: Saving model/pytorch_model-00002-of-00004.bin...
Unsloth: Saving model/pytorch_model-00003-of-00004.bin...
Unsloth: Saving model/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at model into q8_0 GGUF format.
The output location will be /content/model/unsloth.Q8_0.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00004.bin'
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> Q8_0, shape = {4096, 128256}
INFO:hf-to-gguf:blk.0.attn_q.weight,         torch.fl

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Unsloth: Conversion completed! Output location: /content/model/unsloth.Q8_0.gguf


In [ ]:
import subprocess
subprocess.Popen(["ollama", "serve"])
import time
time.sleep(3)

In [ ]:
print(tokenizer._ollama_modelfile)

AttributeError: 'PreTrainedTokenizerFast' object has no attribute '_ollama_modelfile'

In [ ]:
!ollama create unsloth_model -f ./model/Modelfile

transferring model data ⠙ transferring model data ⠹ transferring model data ⠸ transferring model data ⠼ transferring model data ⠴ transferring model data ⠦ transferring model data ⠧ transferring model data ⠇ transferring model data ⠏ transferring model data ⠋ transferring model data ⠙ transferring model data ⠹ transferring model data ⠸ transferring model data ⠼ transferring model data ⠴ transferring model data ⠴ transferring model data ⠧ transferring model data ⠇ transferring model data ⠏ transferring model data ⠋ transferring model data ⠙ transferring model data ⠹ transferring model data ⠸ transferring model data ⠼ transferring model data ⠼ transferring model data ⠦ transferring model data ⠧ transferring model data ⠇ transferring model data ⠏ transferring model data ⠏ transferring model data ⠙ transferring model data ⠹ transferring model data ⠸ transferring model data ⠼ transferring model data ⠴ transferring model data ⠦ transferring model data ⠧ transferring model data ⠇ transferring

In [ ]:
!curl http://localhost:11434/api/chat -d '{ \
    "model": "unsloth_model", \
    "messages": [ \
        { "role": "user", "content": "Who is Haridev?" } \
    ] \
    }'

{"model":"unsloth_model","created_at":"2024-11-01T23:42:47.740052497Z","message":{"role":"assistant","content":" He"},"done":false}
{"model":"unsloth_model","created_at":"2024-11-01T23:42:49.392821478Z","message":{"role":"assistant","content":" has"},"done":false}
{"model":"unsloth_model","created_at":"2024-11-01T23:42:50.84905002Z","message":{"role":"assistant","content":" some"},"done":false}
{"model":"unsloth_model","created_at":"2024-11-01T23:42:52.877800317Z","message":{"role":"assistant","content":" strange"},"done":false}
{"model":"unsloth_model","created_at":"2024-11-01T23:42:54.983582023Z","message":{"role":"assistant","content":" powers"},"done":false}
{"model":"unsloth_model","created_at":"2024-11-01T23:42:56.11113904Z","message":{"role":"assistant","content":" and"},"done":false}
{"model":"unsloth_model","created_at":"2024-11-01T23:42:57.653734271Z","message":{"role":"assistant","content":" abilities"},"done":false}
{"model":"unsloth_model","created_at":"2024-11-01T23:42:58

TILL HERE COMPULSORY

In [ ]:
# Evaluate the model
results = trainer.evaluate(eval_dataset=ClumsyAahDeveloper/sample003)
print(results)

NameError: name 'ClumsyAahDeveloper' is not defined